## Importación de librerias

In [102]:
# !pip install sentiment-analysis-spanish
# !pip install gliner
# !pip install sentence_transformers
# !pip install spacy



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sentiment_analysis_spanish import sentiment_analysis
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import unicodedata

## Clasificacion del estado de animo

In [4]:
def clasificar_estado_animo_con_reglas(frase, resultado=None):
    # Lista de palabras clave para detectar estados negativos
    palabras_negativas = ["llorando", "triste", "deprimido", "desanimado", "solo", "mal", "angustiado", "preocupado", "sufrimiento", "desesperado"]

    # Si detecta palabras negativas, fuerza el estado de ánimo a "Melancólico"
    if any(palabra in frase.lower() for palabra in palabras_negativas):
        return "Melancólico"

    # Verifica si `resultado` es válido antes de intentar acceder a él
    if resultado is not None and len(resultado) > 0:
        label = resultado[0]['label']
        score = resultado[0]['score']

        if label == '5 stars' or (label == '4 stars' and score > 0.8):
            return "Alegre"
        elif label == '3 stars' or (label == '4 stars' and score <= 0.8):
            return "Neutral"
        else:
            return "Melancólico"
    else:
        # Retorna un valor predeterminado si `resultado` es None o vacío
        return "Estado de ánimo desconocido"

In [5]:
sentiment_analyzer = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

Ejemplos de uso

In [6]:
frase_usuario = "ha nacido mi hija"
resultado = sentiment_analyzer(frase_usuario)
estado_animo = clasificar_estado_animo_con_reglas(frase_usuario, resultado)
print(f"Estado de ánimo clasificado: {estado_animo}")

Estado de ánimo clasificado: Alegre


In [5]:
frase_usuario = "me sient bien"
resultado = sentiment_analyzer(frase_usuario)
estado_animo = clasificar_estado_animo_con_reglas(frase_usuario, resultado)
print(f"Estado de ánimo clasificado: {estado_animo}")

Estado de ánimo clasificado: Neutral


In [6]:
frase_usuario = "estoy enfermo"
resultado = sentiment_analyzer(frase_usuario)
estado_animo = clasificar_estado_animo_con_reglas(frase_usuario, resultado)
print(f"Estado de ánimo clasificado: {estado_animo}")

Estado de ánimo clasificado: Melancólico


## Configuración del modelo

In [7]:

# Configuración del modelo de análisis de sentimientos y embeddings
sentiment_analyzer = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')
# Cargar el modelo de embeddings
modelo_embeddings = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


In [8]:
# Cargar datasets
df_peliculas = pd.read_csv('IMDB-Movie-Data.csv')
df_juegos = pd.read_csv('bgg_database.csv')

In [10]:
# !pip install pytesseract
# !pip install pdf2image
# !pip install PyPDF2
# !pip install python-docx
# !pip install SpeechRecognition

import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# Se guarda la URL de los libros
# Se realiza una solicitud HTTP para obtener el contenido de la página
url_libros = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'
response = requests.get(url_libros)
soup = BeautifulSoup(response.text, 'html.parser')

# Se navega por los diferentes "div" de la página para localizar la información de importancia
container_div = soup.find("div", class_="container")
libros_bloque = container_div.find("div", class_="page_content")

libros_links = libros_bloque.find('ol')

# Se crea una lista para almacenar los libros
links = []

# Si se encuentra información dentro de los links de los libros:
if libros_links:
    # Se itera dentro de cada link
    for li in libros_links.find_all("li"):
        # Encuentra la etiqueta donde está contenido el link de los diferentes libros
        a_tag = li.find("a")

        if a_tag and 'href' in a_tag.attrs:
            # Se obtiene el link y luego se le acopla al link base de los diferentes libros
            link = a_tag['href']
            url_cada_libro = 'https://www.gutenberg.org' + link

            # Realiza una solicitud para obtener el contenido de la página individual del libro
            response_libro = requests.get(url_cada_libro)
            soup_libro = BeautifulSoup(response_libro.text, 'html.parser')

            # Navegamos por la información del libro para guardar los datos más relevantes
            container_div_libro = soup_libro.find("div", class_="container")
            pagina_container_div_libro = container_div_libro.find("div", class_="page_content", id='content')
            pagina_body_div = pagina_container_div_libro.find("div", class_="page-body")
            tabs_wrapper_div = pagina_body_div.find("div", id="tabs-wrapper")
            bibrec_div = tabs_wrapper_div.find("div", id="bibrec")
            ebook_div = bibrec_div.find("div", {"typeof": "pgterms:ebook"})
            table = ebook_div.find("table", class_="bibrec")

            # Inicializa estructuras de datos para almacenar información sobre el libro
            datos = []
            informacion = {}
            subjects = []
            libro = {}

            # Extrae cada fila (<tr>) de la tabla y selecciona la etiqueta <th> (clave) y <td> (valor)
            for tr in table.find_all("tr"):
                th = tr.find("th")
                td = tr.find("td")

                if th and td:
                    key = th.text.strip()
                    value = td.text.strip()

                    # Almacena la información específica del libro en el diccionario
                    if key == 'Author':
                        informacion[key] = value
                    if key == 'Title':
                        informacion[key] = value
                    if key == 'Summary':
                        informacion[key] = value
                    if key == 'Subject':
                        subjects.append(value)  # Los temas se almacenan en una lista ya que para cada libro existe más de un tópico

            # Añade la lista de temas al diccionario de información del libro
            informacion['Subjects'] = subjects
            links.append(informacion)

            # Pausa de 1 segundo entre solicitudes para evitar el rechazo del servidor
            time.sleep(1)

# Se crea un DataFrame con la información guardada
# Se separa por "#" para evitar posibles problemas en un futuro
todos_libros = pd.DataFrame(links)
todos_libros.to_csv('todos_libros.csv', index=False, sep='#')

# Se descarga el DataFrame para que no sea necesario correr el código cada vez que se solicite la información
from google.colab import files
files.download("todos_libros.csv")

[{'Credits': 'Produced by Don Kostuch', 'Language': 'English', 'LoC Class': 'HS: Social sciences: Societies: secret, benevolent, etc.', 'Subjects': ['Boy Scouts of America -- Handbooks, manuals, etc.', 'Boy Scouts -- United States -- Handbooks, manuals, etc.'], 'Category': 'Text', 'EBook-No.': '29558', 'Release Date': 'Aug 1, 2009', 'Most Recently Updated': 'Jan 5, 2021', 'Copyright Status': 'Public domain in the USA.', 'Downloads': '1424 downloads in the last 30 days.'}]


In [9]:
# Se separa por "#" para evitar posibles problemas en un futuro
df_libros = pd.DataFrame(links)
df_libros.to_csv('libros.csv', index=False, sep='#')

# # Se descarga el DataFrame para que no sea necesario correr el código cada vez que se solicite la información
# from google.colab import files
# files.download("libros.csv")

NameError: name 'links' is not defined

In [12]:
links
# # Convertir la lista de diccionarios a un DataFrame
# df_libros = pd.DataFrame(links)

# # Guardar el DataFrame en un archivo CSV
# df_libros.to_csv('libros_gutenberg.csv', index=False, encoding='utf-8')

# print("Datos guardados en 'libros_gutenberg.csv'")

[{'Credits': 'Produced by Don Kostuch',
  'Language': 'English',
  'LoC Class': 'HS: Social sciences: Societies: secret, benevolent, etc.',
  'Subjects': ['Boy Scouts of America -- Handbooks, manuals, etc.',
   'Boy Scouts -- United States -- Handbooks, manuals, etc.'],
  'Category': 'Text',
  'EBook-No.': '29558',
  'Release Date': 'Aug 1, 2009',
  'Most Recently Updated': 'Jan 5, 2021',
  'Copyright Status': 'Public domain in the USA.',
  'Downloads': '1424 downloads in the last 30 days.'}]

In [10]:
df_peliculas

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,0.00,45
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,0.00,22


Para mejorar el rendimiento y reducir el tiempo de procesamiento, podemos optimizar la generación de los embeddings (vectores) para los títulos de películas, juegos y libros. En lugar de calcularlos cada vez que ejecutamos el programa, es más eficiente guardarlos una vez y luego cargarlos.

In [11]:
# Modelo de embeddings
modelo_embeddings = SentenceTransformer('all-mpnet-base-v2')

# Función para guardar los vectores embeddings en un archivo .pkl
def guardar_embeddings(titulos, filename):
    vectores = [modelo_embeddings.encode(titulo) for titulo in titulos]
    with open(filename, 'wb') as f:
        pickle.dump(vectores, f)

# Función para cargar los vectores embeddings desde un archivo .pkl
def cargar_embeddings(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

# Guardar vectores embeddings si no existen archivos .pkl
import os

# Películas
if not os.path.exists("vector_peliculas.pkl"):
    titulos_peliculas = df_peliculas['Title'].tolist()
    guardar_embeddings(titulos_peliculas, "vector_peliculas.pkl")
vector_peliculas = cargar_embeddings("vector_peliculas.pkl")

# Juegos
if not os.path.exists("vector_juegos.pkl"):
    titulos_juegos = df_juegos['game_name'].tolist()
    guardar_embeddings(titulos_juegos, "vector_juegos.pkl")
vector_juegos = cargar_embeddings("vector_juegos.pkl")

# Libros
if not os.path.exists("vector_libros.pkl"):
    titulos_libros = df_libros['Title'].tolist()
    guardar_embeddings(titulos_libros, "vector_libros.pkl")
vector_libros = cargar_embeddings("vector_libros.pkl")


In [12]:
print(titulos_peliculas)  # Verifica que esta variable tenga datos
print(titulos_juegos)     # Verifica que esta variable tenga datos
print(titulos_libros)     # Verifica que esta variable tenga datos

['Guardians of the Galaxy', 'Prometheus', 'Split', 'Sing', 'Suicide Squad', 'The Great Wall', 'La La Land', 'Mindhorn', 'The Lost City of Z', 'Passengers', 'Fantastic Beasts and Where to Find Them', 'Hidden Figures', 'Rogue One', 'Moana', 'Colossal', 'The Secret Life of Pets', 'Hacksaw Ridge', 'Jason Bourne', 'Lion', 'Arrival', 'Gold', 'Manchester by the Sea', 'Hounds of Love', 'Trolls', 'Independence Day: Resurgence', 'Paris pieds nus', 'Bahubali: The Beginning', 'Dead Awake', 'Bad Moms', "Assassin's Creed", 'Why Him?', 'Nocturnal Animals', 'X-Men: Apocalypse', 'Deadpool', 'Resident Evil: The Final Chapter', 'Captain America: Civil War', 'Interstellar', 'Doctor Strange', 'The Magnificent Seven', '5/25/1977', 'Sausage Party', 'Moonlight', "Don't Fuck in the Woods", 'The Founder', 'Lowriders', 'Pirates of the Caribbean: On Stranger Tides', 'Miss Sloane', 'Fallen', 'Star Trek Beyond', 'The Last Face', 'Star Wars: Episode VII - The Force Awakens', 'Underworld: Blood Wars', "Mother's Day",

Para comparar la preferencia del usuario con los títulos de las recomendaciones, usaremos la similitud de coseno, optimizando su cálculo al estructurar los datos para mejorar la precisión y eficiencia.

In [1]:
# Paso 1: Ingreso de Preferencias
frase_usuario = input("Ingresa una frase que describa la temática que te gustaría explorar: ")

# Normalizar la frase ingresada
frase_usuario_normalizada = frase_usuario.lower().strip()

# Paso 2: Búsqueda de Opciones
def buscar_opciones(df, frase):
    # Generar el embedding de la frase ingresada
    vector_frase = modelo_embeddings.encode(frase)
    
    # Calcular la similitud con los títulos en el DataFrame
    df['Similaridad'] = df['Title'].apply(lambda x: cosine_similarity([vector_frase], [modelo_embeddings.encode(x)])[0][0])
    
    # Devolver las 5 opciones más similares
    return df.nlargest(5, 'Similaridad')

# Buscar opciones en libros, películas y juegos
opciones_libros = buscar_opciones(df_libros, frase_usuario_normalizada)
opciones_peliculas = buscar_opciones(df_peliculas, frase_usuario_normalizada)
opciones_juegos = buscar_opciones(df_juegos, frase_usuario_normalizada)

# Mostrar resultados
print("\nOpciones de Libros:")
print(opciones_libros[['Title', 'Similaridad']])

print("\nOpciones de Películas:")
print(opciones_peliculas[['Title', 'Similaridad']])

print("\nOpciones de Juegos de Mesa:")
print(opciones_juegos[['game_name', 'Similaridad']])

NameError: name 'df_libros' is not defined

In [ ]:
# Paso 1: Cargar los embeddings
vector_peliculas = cargar_embeddings("vector_peliculas.pkl")
vector_juegos = cargar_embeddings("vector_juegos.pkl")
vector_libros = cargar_embeddings("vector_libros.pkl")

# Paso 2: Ingreso de Preferencias
frase_usuario = input("Ingresa una frase que describa la temática que te gustaría explorar: ")

# Normalizar la frase ingresada
frase_usuario_normalizada = frase_usuario.lower().strip()

# Paso 3: Identificar el tipo de contenido
tipo_contenido = None
if "película" in frase_usuario_normalizada:
    tipo_contenido = "pelicula"
elif "libro" in frase_usuario_normalizada:
    tipo_contenido = "libro"
elif "juego de mesa" in frase_usuario_normalizada:
    tipo_contenido = "juego"

# Paso 4: Búsqueda de Opciones
def buscar_opciones(df, frase):
    # Generar el embedding de la frase ingresada
    vector_frase = modelo_embeddings.encode(frase)
    
    # Calcular la similitud con los títulos en el DataFrame
    df['Similaridad'] = df['Title'].apply(lambda x: cosine_similarity([vector_frase], [modelo_embeddings.encode(x)])[0][0])
    
    # Devolver las 5 opciones más similares
    return df.nlargest(5, 'Similaridad')

# Buscar opciones según el tipo de contenido
if tipo_contenido == "pelicula":
    opciones = buscar_opciones(df_peliculas, frase_usuario_normalizada)
    print("\nOpciones de Películas:")
    print(opciones[['Title', 'Similaridad']])
elif tipo_contenido == "libro":
    opciones = buscar_opciones(df_libros, frase_usuario_normalizada)
    print("\nOpciones de Libros:")
    print(opciones[['Title', 'Similaridad']])
elif tipo_contenido == "juego":
    opciones = buscar_opciones(df_juegos, frase_usuario_normalizada)
    print("\nOpciones de Juegos de Mesa:")
    print(opciones[['game_name', 'Similaridad']])
else:
    print("No se reconoció el tipo de contenido. Por favor, incluye 'película', 'libro' o 'juego de mesa' en tu frase.")

Extraccion de entidades

Generar embedding para texto de input

In [13]:
def obtener_embedding(texto):
    """
    Genera el embedding para un texto dado usando el modelo Sentence Transformers.
    """
    return modelo_embeddings.encode([texto])[0]

def eliminar_acentos(texto):
    """
    Elimina acentos de un texto.
    """
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )


Deteccion de categorias

In [14]:
def detectar_categoria(preferencia):
    """
    Detecta la categoría en la preferencia del usuario, como 'película', 'juego' o 'libro',
    ignorando los acentos.
    """
    preferencia = eliminar_acentos(preferencia.lower())
    if "pelicula" in preferencia:
        return "Películas"
    elif "juego" in preferencia:
        return "Juegos de Mesa"
    elif "libro" in preferencia:
        return "Libros"
    else:
        return None  # Si no se detecta una categoría específica

Generacion de recomendaciones

In [15]:
def obtener_recomendaciones(preferencia, titulos, tipo, top_n=5):
    """
    Genera recomendaciones basadas en la similaridad de coseno entre el embedding
    de la preferencia del usuario y los títulos disponibles.
    """
    # Generar embedding de la preferencia del usuario
    embedding_usuario = obtener_embedding(preferencia)
    
    # Generar embeddings de los títulos
    embeddings_titulos = [obtener_embedding(titulo) for titulo in titulos]
    
    # Calcular similaridad de coseno entre la preferencia del usuario y cada título
    similitudes = cosine_similarity([embedding_usuario], embeddings_titulos)[0]
    
    # Obtener los índices de los títulos con mayor similaridad
    indices_top = similitudes.argsort()[-top_n:][::-1]
    recomendaciones = [titulos[i] for i in indices_top]
    
    # Mostrar recomendaciones
    print(f"\nRecomendaciones de {tipo} para '{preferencia}':")
    for i, recomendacion in enumerate(recomendaciones, 1):
        print(f"{i}. {recomendacion}")

In [16]:
# Flujo principal
# Solicitar al usuario su estado de ánimo
frase_usuario = input("¿Cómo te sientes hoy? ")
resultado = sentiment_analyzer(frase_usuario)  # Analizar la frase del usuario
estado_animo = clasificar_estado_animo_con_reglas(frase_usuario, resultado)  # Clasificar el estado de ánimo
print(f"Estado de ánimo clasificado: {estado_animo}")

# Solicitar la preferencia temática del usuario
preferencia_usuario = input("Describe la temática que te gustaría explorar: ")

# Detectar la categoría basada en la preferencia del usuario
categoria_deseada = detectar_categoria(preferencia_usuario)

# Mostrar recomendaciones según la categoría detectada
if categoria_deseada == "Películas":
    obtener_recomendaciones(preferencia_usuario, titulos_peliculas, "Películas")
elif categoria_deseada == "Juegos de Mesa":
    obtener_recomendaciones(preferencia_usuario, titulos_juegos, "Juegos de Mesa")
elif categoria_deseada == "Libros":
    obtener_recomendaciones(preferencia_usuario, titulos_libros, "Libros")
else:
    # Si no se detecta ninguna categoría específica, se muestran todas las recomendaciones
    print("\nNo se detectó una categoría específica. Mostrando todas las recomendaciones:")
    obtener_recomendaciones(preferencia_usuario, titulos_peliculas, "Películas")
    obtener_recomendaciones(preferencia_usuario, titulos_juegos, "Juegos de Mesa")
    obtener_recomendaciones(preferencia_usuario, titulos_libros, "Libros")

Estado de ánimo clasificado: Alegre

Recomendaciones de Películas para 'pelicula de terror':
1. La tortue rouge
2. Bastille Day
3. Planet Terror
4. L'avenir
5. Ma vie de Courgette


In [17]:
import spacy

# Cargar el modelo en español de spaCy
nlp = spacy.load("es_core_news_md")  # Asegúrate de tener el modelo `es_core_news_md` descargado

# Función para obtener el embedding de un género
def obtener_embedding(texto):
    return nlp(texto).vector


In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Supongamos que tienes una columna "Genre" en el DataFrame de películas (df_peliculas)
# y que contiene los géneros que quieres comparar.

# Calcular embeddings para cada género en tu dataset
df_peliculas['Genre_embedding'] = df_peliculas['Genre'].apply(lambda x: obtener_embedding(x))

# Función para encontrar las recomendaciones más similares en género
def recomendaciones_similares(df_peliculas, input_usuario, umbral=0.5):
    # Obtener embedding del input del usuario
    input_embedding = obtener_embedding(input_usuario)
    
    # Calcular similitud de coseno entre el input y cada género en el DataFrame
    df_peliculas['Similaridad'] = df_peliculas['Genre_embedding'].apply(lambda x: cosine_similarity([input_embedding], [x])[0][0])
    
    # Filtrar recomendaciones basadas en un umbral de similitud
    recomendaciones = df_peliculas[df_peliculas['Similaridad'] >= umbral]
    recomendaciones_ordenadas = recomendaciones.sort_values(by="Similaridad", ascending=False)
    
    return recomendaciones_ordenadas[['Title', 'Genre', 'Similaridad']].head(5)  # Devuelve las 5 más similares


In [29]:
# Calcular embeddings para la columna de géneros o descripciones en el DataFrame de libros
df_libros['Genre_embedding'] = df_libros['Subjects'].apply(lambda x: obtener_embedding(x))

# Función para encontrar las recomendaciones de libros similares en género o descripción
def recomendaciones_libros_similares(df_libros, input_usuario, umbral=0.5):
    # Obtener embedding del input del usuario
    input_embedding = obtener_embedding(input_usuario)
    
    # Calcular similaridad del coseno entre el input y cada género o descripción en el DataFrame
    df_libros['Similaridad'] = df_libros['Genre_embedding'].apply(lambda x: cosine_similarity([input_embedding], [x])[0][0])
    
    # Filtrar recomendaciones basadas en un umbral de similitud
    recomendaciones = df_libros[df_libros['Similaridad'] >= umbral]
    recomendaciones_ordenadas = recomendaciones.sort_values(by="Similaridad", ascending=False)
    
    return recomendaciones_ordenadas[['Title', 'Subjects', 'Similaridad']].head(5)  # Devuelve las 5 más similares


In [32]:
# Calcular embeddings para la columna de categorías o descripciones en el DataFrame de juegos
df_juegos['Category_embedding'] = df_juegos['categories'].apply(lambda x: obtener_embedding(x))

# Función para encontrar recomendaciones de juegos de mesa similares en categoría o descripción
def recomendaciones_juegos_similares(df_juegos, input_usuario, umbral=0.5):
    # Obtener embedding del input del usuario
    input_embedding = obtener_embedding(input_usuario)
    
    # Calcular similaridad del coseno entre el input y cada categoría o descripción en el DataFrame
    df_juegos['Similaridad'] = df_juegos['Category_embedding'].apply(lambda x: cosine_similarity([input_embedding], [x])[0][0])
    
    # Filtrar recomendaciones basadas en un umbral de similitud
    recomendaciones = df_juegos[df_juegos['Similaridad'] >= umbral]
    recomendaciones_ordenadas = recomendaciones.sort_values(by="Similaridad", ascending=False)
    
    return recomendaciones_ordenadas[['game_name', 'categories', 'Similaridad']].head(5)  # Devuelve las 5 más similares


In [40]:
def recomendaciones_libros_similares(df, input_usuario):
    # Aquí deberías calcular la similitud entre el input del usuario y los títulos en df
    # Por ejemplo, usando embeddings o alguna otra técnica de similitud
    # Este es un ejemplo básico que puedes ajustar según tu lógica
    input_vector = modelo_embeddings.encode(input_usuario)
    df['Similaridad'] = df['Title'].apply(lambda x: cosine_similarity([input_vector], [modelo_embeddings.encode(x)])[0][0])
    return df.nlargest(5, 'Similaridad')  # Devuelve las 5 recomendaciones más similares

def recomendaciones_juegos_similares(df, input_usuario):
    input_vector = modelo_embeddings.encode(input_usuario)
    df['Similaridad'] = df['game_name'].apply(lambda x: cosine_similarity([input_vector], [modelo_embeddings.encode(x)])[0][0])
    return df.nlargest(5, 'Similaridad')  # Devuelve las 5 recomendaciones más similares

In [43]:
# Flujo principal
# Solicitar al usuario su estado de ánimo
frase_usuario = input("¿Cómo te sientes hoy? ")
resultado = sentiment_analyzer(frase_usuario)  # Analizar la frase del usuario
estado_animo = clasificar_estado_animo_con_reglas(frase_usuario, resultado)  # Clasificar el estado de ánimo
print(f"Estado de ánimo clasificado: {estado_animo}")

# Paso 1: Cargar los embeddings
vector_peliculas = cargar_embeddings("vector_peliculas.pkl")
vector_juegos = cargar_embeddings("vector_juegos.pkl")
vector_libros = cargar_embeddings("vector_libros.pkl")

# Paso 2: Ingreso de Preferencias
frase_usuario = input("Ingresa una frase que describa la temática que te gustaría explorar: ")

# Normalizar la frase ingresada
frase_usuario_normalizada = frase_usuario.lower().strip()

# Paso 3: Identificar el tipo de contenido
tipo_contenido = None
frase_usuario_sin_acentos = frase_usuario_normalizada  # Aquí puedes usar la función eliminar_acentos si no quieres usar unidecode

if "pelicula" in frase_usuario_sin_acentos:
    tipo_contenido = "pelicula"
elif "libro" in frase_usuario_sin_acentos:
    tipo_contenido = "libro"
elif "juego de mesa" in frase_usuario_sin_acentos:
    tipo_contenido = "juego"

# Paso 4: Búsqueda de Opciones
def buscar_opciones(df, frase, tipo):
    # Generar el embedding de la frase ingresada por el usuario
    vector_frase = modelo_embeddings.encode(frase)
    
    # Concatenar los campos de interés según el tipo de contenido
    if tipo == "pelicula":
        df['Texto_completo'] = df['Title'].fillna('') + " " + df.get('Description', '').fillna('') + " " + df.get('Genre', '').fillna('')
    elif tipo == "libro":
        df['Texto_completo'] = df['Title'].fillna('') + " " + df.get('Summary', '').fillna('') + " " + df.get('Subjects', '').fillna('')
    elif tipo == "juego":
        # Asegurarse de que 'categories' sea una cadena
        df['Texto_completo'] = df['game_name'].fillna('') + " " + df.get('categories', '').apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x)).fillna('')

    # Calcular la similitud con el texto completo en cada fila del DataFrame
    df['Similaridad'] = df['Texto_completo'].apply(lambda x: cosine_similarity([vector_frase], [modelo_embeddings.encode(x)])[0][0])
    
    # Devolver las 5 opciones más similares
    return df.nlargest(5, 'Similaridad')

# Buscar opciones según el tipo de contenido
if tipo_contenido == "pelicula":
    opciones = buscar_opciones(df_peliculas, frase_usuario_normalizada, tipo_contenido)
    print("\nOpciones de Películas:")
    print(opciones[['Title', 'Description', 'Genre', 'Similaridad']])
elif tipo_contenido == "libro":
    opciones = buscar_opciones(df_libros, frase_usuario_normalizada, tipo_contenido)
    print("\nOpciones de Libros:")
    print(opciones[['Title', 'Summary', 'Subjects', 'Similaridad']])
elif tipo_contenido == "juego":
    opciones = buscar_opciones(df_juegos, frase_usuario_normalizada, tipo_contenido)
    print("\nOpciones de Juegos de Mesa:")
    print(opciones[['game_name', 'Genre', 'Similaridad']])
else:
    print("No se reconoció el tipo de contenido. Por favor, incluye 'película', 'libro' o 'juego de mesa' en tu frase.")

KeyboardInterrupt: 

In [38]:
# # Ejemplo de input
# input_usuario = "película de diversion"
# recomendaciones = recomendaciones_similares(df_peliculas, input_usuario)
# print(recomendaciones)
# Ejemplo de input para libros y juegos de mesa

Recomendaciones de Libros:
Empty DataFrame
Columns: [Title, Subjects, Similaridad]
Index: []

Recomendaciones de Juegos de Mesa:
Empty DataFrame
Columns: [game_name, categories, Similaridad]
Index: []
